In [24]:
import pinocchio as pin
from pinocchio.utils import *

import hppfcl

import example_robot_data as robex
import numpy as np
from numpy.linalg import inv, pinv, eig, norm, svd, det
from scipy.optimize import fmin_bfgs
import time

import vizutils
import random
import math

import H_RRT as rrt
import H_PathTraversal as pt
import H_Utils as ut




In [2]:
robot = robex.loadSolo()

rmodel = robot.model
rdata = rmodel.createData()

rcollision_model = robot.collision_model
rcollision_data  = rcollision_model.createData()

Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, rcollision_model, robot.visual_model)
viz.initViewer(loadModel=True)



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()


# --- ENVIRONMENT DEFINITION ----------------------------------------

In [4]:
viz.display(robot.q0)


In [5]:
# Load collision geometries


In [6]:


'''  
deg = 1.76398 / 2
quat = [math.cos(deg),math.sin(deg),0,0]
quat = np.array(quat,np.float64)

print(quat)



d = norm(quat)
quat = quat/d
print(quat)

xyzq = np.array([3, 3, .2])


xyzq = np.concatenate([xyzq,quat])
print(xyzq)

''' 

vizutils.addViewerBox(viz, 'world/floor', 12, 12, .1, [.8, .8, .8, .41])
vizutils.applyViewerConfiguration(viz, 'world/floor', [0, 0, -.01, 1, 0, 0, 0])



In [7]:
def XYZtoQuat(x,y,angle):
    
    angle = angle / 2
    quat = [math.cos(angle),math.sin(angle),0,0]
    quat = np.array(quat,np.float64)
    quat = quat/norm(quat)
    
    q = np.array([x, y, .2])
    q = np.concatenate([q,quat])    
    return q
    

In [69]:
#def angleSE3(x, y, z, axis, angle):

class Obstacle:
    def __init__(self,name,xyzquat,sizex, sizey, sizez, angle=None):
        self.name = 'world/obs/' + str(name)
        self.xyzquat = xyzquat
        self.sizex = sizex
        self.sizey = sizey
        self.sizez = sizez
        self.angle = angle if angle is not None else 0
        self.Mobs  = ut.angleSE3(xyzquat[0],xyzquat[1],xyzquat[2],[0,0,1],self.angle)
        print(self.xyzquat)
        
        vizutils.addViewerBox(viz, self.name, sizex, sizey, sizez, [1., 1, .2, 1])
        vizutils.applyViewerConfiguration(viz, self.name, self.xyzquat)
        
        #create colision object:
        shape = hppfcl.Box(1, 1, 1)
        self.geom  = hppfcl.CollisionObject(self.name, 0,shape, self.Mobs)
        print(self.geom)
        
        
        
        
cObs = []

aObs = 0                        #angle in z
qObs = XYZtoQuat(1.6, 4,aObs)     #position x,y
obs  = Obstacle('box1',qObs,1, .5, .4,aObs)
cObs.append(obs)


aObs = 0.5
qObs = XYZtoQuat(2.3, 1.3 ,aObs)
obs  = Obstacle('box2',qObs,.5, 1, .4,aObs)
cObs.append(obs)

aObs = 0.7
qObs = XYZtoQuat(4, 3,aObs)
obs  = Obstacle('box3',qObs,.7, .7, .4,aObs)
cObs.append(obs)


print(hppfcl.distance(cObs[0].geom,cObs[1].geom))

[1.6 4.  0.2 1.  0.  0.  0. ]


AttributeError: module 'hppfcl' has no attribute 'CollisionObject'

In [46]:
class RootBox:
    def __init__(self,q):
        self.qBox = q.copy()[:7]
        print(self.qBox)
        self.xSize = .43
        self.ySize = .26
        self.zSize = .15
        vizutils.applyViewerConfiguration(viz, 'world/root', self.qBox)
        vizutils.addViewerBox(viz, 'world/root', self.xSize, self.ySize, self.zSize, [1., .2, .2, .5])
        #create initial colision object (probably useless)
        
        self.angle = 0
        self.Mobs  = ut.angleSE3(self.qBox[0],self.qBox[1],self.qBox[2],[0,0,1],0)
        print(self.Mobs)

        
    def alpha(self,alph):
        self.alph = alph + 0.0
        vizutils.addViewerBox(viz, 'world/root', .43, .26, .15, [1., .2, .2, alph])
    
    def move(self, q):
        self.qBox  = q.copy()[:7]
        vizutils.applyViewerConfiguration(viz, 'world/root', self.qBox)

    def isValidPosition(self,XY,angle,cObs):
        qCheck = XYZtoQuat(XY[0],XY[1],angle)
        vizutils.applyViewerConfiguration(viz, 'world/root', qCheck)
        
        time.sleep(0.1)
        #for obs in cObs:
            ##check collision ;D    
            
        vizutils.applyViewerConfiguration(viz, 'world/root', self.qBox)    
        return True
    
    
    
    
robotReach = RootBox(robot.q0)
robotReach.alpha(0.5)


[0.    0.    0.235 0.    0.    0.    1.   ]
  R =
1 0 0
0 1 0
0 0 1
  p =     0     0 0.235



## RRT*


In [ ]:
#constants
XDIM = 5
YDIM = 5
WINSIZE = [XDIM, YDIM]
EPSILON = 0.3
NUMNODES = 1000
RADIUS=1.3

## Start Here:

In [ ]:
#setup run:
BUFFER_WIDTH = .26 / 2
OPT_RADIUS = 0.3

qguess = robot.q0.copy()
viz.display(qguess)

print(qguess[:2])
viz.viewer['world/nodes'].delete() #Deletes all figures in /dir


In [ ]:


goalXY     =[random.random()*XDIM, random.random()*YDIM]
inObstacle = rrt.checkCobs(goalXY,cObs,BUFFER_WIDTH)

if not inObstacle[0]: 
    vizutils.addViewerSphere(viz, 'world/nodes/end', .04, [.2, 1, .6, 1])
    vizutils.applyViewerConfiguration(viz, 'world/nodes/end', [goalXY[0],goalXY[1],.2, 1, 0, 0, 0])  
    


In [ ]:

print("-searching")
nodes,path = rrt.RRT(qguess[:2],goalXY,BUFFER_WIDTH,cObs,robotReach,viz)

print("- path nodes: " + str(len(path)))


In [ ]:

CLUSTER_RADIUS = .3
clustered_path = rrt.clusterPath(path,CLUSTER_RADIUS,viz)
print(len(clustered_path))

In [ ]:
#now we check collision along the subpaths, and correct any corner clipping 
root_path = []


root_path = rrt.correctPath(clustered_path,cObs,BUFFER_WIDTH)


## PATH FOLLOWING

#print(XYZQUATToSe3(qguess[:7]))
    quat = pin.Quaternion.Identity()
    quat[0] = qguess[3]
    quat[1] = qguess[4]
    quat[2] = qguess[5]
    quat[3] = qguess[6]
    
    
    print(quat)

    Mq = pin.SE3(quat,qguess[:3])
    
    print(Mq)

In [ ]:
#cost = Cost3d(rmodel, rdata, frameIndex = 1, ptarget= robot.q0[:3], viz=viz)
#qguess = robot.q0.copy()

root_path.reverse()

for obj in root_path:
    segments = []
    segments = pt.genSegments(qguess, obj.position(),viz)
    qguess = pt.travelSegments(segments,qguess,rmodel, rdata,robot,robotReach,viz)
    qguess = pin.normalize(rmodel, qguess)  # Now, it is normalize

    obj.remove()

In [ ]:
Deprecated:

In [ ]:

        #axis = np.array([math.cos(deg),math.sin(deg),0,0],np.float64)
        #axis /= norm(axis)
        
        #quat = pin.Quaternion.Identity()
        #quat[0] = axis[2]
        #quat[1] = axis[3]
        #quat[2] = axis[1]
        #quat[3] = axis[0]
        #print(quat)
        
        ##MtargetEnd = pin.SE3(quat,np.array(seg.endq[:3]))
        
se = ut.angleSE3(0, 0, .23, [0,0,1], 1.76398)

print(se)

print(se.rotation)